In [7]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split

# === Load .npy files ===
X_seq = np.load("X_seq.npy")  # shape: (N, 100, 1024)
y_seq = np.load("y_seq.npy")  # shape: (N,)

# === Split into train/test (stratified) ===
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_seq, y_seq, test_size=0.2, stratify=y_seq, random_state=42
)

# === Convert to PyTorch tensors ===
X_train = torch.tensor(X_train_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.long)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_test = torch.tensor(y_test_np, dtype=torch.long)

# === Wrap in TensorDataset ===
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# === Split train into train/val ===
val_ratio = 0.1
val_size = int(len(train_dataset) * val_ratio)
train_size = len(train_dataset) - val_size
train_set, val_set = random_split(train_dataset, [train_size, val_size])

# === Build DataLoaders ===
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

print("✅ DataLoaders ready:")
print(f"Train: {len(train_set)} | Val: {len(val_set)} | Test: {len(test_dataset)}")


✅ DataLoaders ready:
Train: 7727 | Val: 858 | Test: 2147


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class YAMNet1DCNN_Improved(nn.Module):
    def __init__(self, num_classes=57):
        super().__init__()
        self.conv1 = nn.Conv1d(1024, 512, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(512)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(512, 256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(256)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(256, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.75)  # Slightly higher for better regularization
        self.fc = nn.Linear(128, num_classes)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (B, 100, 1024) → (B, 1024, 100)
        x = self.pool1(F.gelu(self.bn1(self.conv1(x))))
        x = self.pool2(F.gelu(self.bn2(self.conv2(x))))
        x = F.gelu(self.bn3(self.conv3(x)))
        x = self.global_pool(x).squeeze(-1)
        x = self.dropout(x)
        return self.fc(x)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

class YAMNet1DCNN_Improved(nn.Module):
    def __init__(self, num_classes=57):
        super().__init__()
        self.conv1 = nn.Conv1d(1024, 512, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(512)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(512, 256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(256)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(256, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.75)  # Slightly higher for better regularization
        self.fc = nn.Linear(128, num_classes)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (B, 100, 1024) → (B, 1024, 100)
        x = self.pool1(F.gelu(self.bn1(self.conv1(x))))
        x = self.pool2(F.gelu(self.bn2(self.conv2(x))))
        x = F.gelu(self.bn3(self.conv3(x)))
        x = self.global_pool(x).squeeze(-1)
        x = self.dropout(x)
        return self.fc(x)


model = YAMNet1DCNN_Improved()
model.load_state_dict(torch.load("Best_Model.pt"))
model.eval()
torch.save(model, "yamnet_model_cnn.pt")  # saves model architecture + weights

example_input = torch.randn(1, 100, 1024)
traced_model = torch.jit.trace(model, example_input)
traced_model.save("Mobile_Audio_Classifier.pt")



In [ ]:
%pip install kivy soundfile torch

In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score  # ✅ Needed for final test acc
from tqdm import tqdm  
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YAMNet1DCNN_Improved().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=4,
    min_lr=1e-6
)

best_val_acc = 0
patience = 200
no_improve_epochs = 0
saved_model_name = None

for epoch in range(1, 201):
    model.train()
    total_loss = 0
    correct = 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (preds.argmax(1) == yb).sum().item()

    train_acc = 100. * correct / len(train_loader.dataset)

    # 🔍 Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            val_correct += (preds.argmax(1) == yb).sum().item()
            val_total += yb.size(0)
    val_acc = 100. * val_correct / val_total

    scheduler.step(val_acc)

    print(f"Epoch {epoch}: Loss={total_loss:.2f}, Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")

    # ✅ Save model if val_acc >= 85 (allow duplicates with epoch number)
    if val_acc >= 85:
        val_str = f"{val_acc:.2f}".replace('.', '_')
        saved_model_name = f"Model_{val_str}_Epoch{epoch}.pt"
        torch.save(model.state_dict(), saved_model_name)
        print(f"✅ Saved model: {saved_model_name} (Val Acc: {val_acc:.2f}%)")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("⛔ Early stopping triggered.")
            break

model.eval()
all_preds = []
with torch.no_grad():
    for xb, _ in test_loader:
        xb = xb.to(device)
        preds = model(xb)
        all_preds.append(preds.cpu())

all_preds = torch.cat(all_preds).argmax(1).numpy()
acc = accuracy_score(y_test, all_preds)

print(f"\n✅ Final Test Accuracy: {acc * 100:.2f}%")

Epoch 1: Loss=530.70, Train Acc=53.81%, Val Acc=65.73%
Epoch 2: Loss=325.42, Train Acc=69.43%, Val Acc=69.70%
Epoch 3: Loss=260.73, Train Acc=73.48%, Val Acc=74.36%
Epoch 4: Loss=223.67, Train Acc=77.21%, Val Acc=77.27%
Epoch 5: Loss=192.62, Train Acc=79.69%, Val Acc=77.86%
Epoch 6: Loss=166.74, Train Acc=82.36%, Val Acc=82.28%
Epoch 7: Loss=152.99, Train Acc=84.03%, Val Acc=82.52%
Epoch 8: Loss=132.73, Train Acc=86.55%, Val Acc=84.27%
Epoch 9: Loss=121.98, Train Acc=87.77%, Val Acc=84.85%
Epoch 10: Loss=115.19, Train Acc=88.25%, Val Acc=86.71%
✅ Saved model: Model_86_71_Epoch10.pt (Val Acc: 86.71%)
Epoch 11: Loss=105.18, Train Acc=89.34%, Val Acc=86.36%
✅ Saved model: Model_86_36_Epoch11.pt (Val Acc: 86.36%)
Epoch 12: Loss=93.01, Train Acc=90.40%, Val Acc=86.48%
✅ Saved model: Model_86_48_Epoch12.pt (Val Acc: 86.48%)
Epoch 13: Loss=85.11, Train Acc=90.85%, Val Acc=87.30%
✅ Saved model: Model_87_30_Epoch13.pt (Val Acc: 87.30%)
Epoch 14: Loss=82.15, Train Acc=91.29%, Val Acc=86.83%
✅ Sa